<a href="https://colab.research.google.com/github/ilya-kkk/ml_trainings_3/blob/main/hw5_autoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Устанавливаем H2O
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o


Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 MB 4.7 MB/s eta 0:00:00


In [3]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

# Запуск H2O сервера
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.26" 2025-01-21; OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpjtsx2p2m
  JVM stdout: /tmp/tmpjtsx2p2m/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpjtsx2p2m/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,30 days
H2O_cluster_name:,H2O_from_python_unknownUser_mlvujl
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [4]:
# Загружаем обучающий и тестовый датасеты
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

# Посмотрим на данные
train.head()


,area,perimeter,major_axis,minor_axis,eccentricity,eqdiasq,solidity,convex_area,extent,aspect_ratio,roundness,compactness,shapefactor_1,shapefactor_2,shapefactor_3,shapefactor_4,target
0,75516,1731.4840,411.7352,245.7620,0.8023,310.0806,0.9148,82546,0.7169,1.6753,0.3165,0.7531,0.0055,0.0033,0.5672,0.9502,1
1,98903,1374.4370,477.2451,269.7676,0.8249,354.8622,0.9585,103181,0.7679,1.7691,0.6579,0.7436,0.0048,0.0027,0.5529,0.9781,0
2,84746,1311.1570,482.7735,235.9040,0.8725,328.4843,0.9121,92914,0.7162,2.0465,0.6195,0.6804,0.0057,0.0028,0.4630,0.9474,1
3,98184,1463.1680,434.3769,292.6472,0.7390,353.5700,0.9543,102890,0.7316,1.4843,0.5763,0.8140,0.0044,0.0030,0.6625,0.9834,0
4,94170,1267.7271,440.1109,278.4162,0.7745,346.2672,0.9643,97656,0.6836,1.5808,0.7363,0.7868,0.0047,0.0030,0.6190,0.9785,0


In [5]:
# Переводим pandas в формат H2O
train_h2o = h2o.H2OFrame(train)
test_h2o = h2o.H2OFrame(test)

# Определяем целевую переменную и признаки
target = 'target'  # или другое имя целевой колонки, если у вас оно отличается
features = train.columns.drop(target).tolist()

# Убедимся, что целевая переменная — категориальная
train_h2o[target] = train_h2o[target].asfactor()


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Запуск AutoML
aml = H2OAutoML(max_runtime_secs=1800,
                max_models=50,
                seed=42,
                nfolds=20,
                sort_metric='accuracy',
                balance_classes=False,
                include_algos=["GBM", "XGBoost", "StackedEnsemble"],  # только бустинги и ансамбли деревьев
                stopping_rounds=15,           # если 5 раз подряд нет улучшений - стоп
                stopping_tolerance=0.001
                )

aml.train(x=features, y=target, training_frame=train_h2o)

# Посмотрим лучшие модели
aml.leaderboard


AutoML progress: |
11:14:24.700: Stopping tolerance set by the user is < 70% of the recommended default of 0.02786391062876764, so models may take a long time to converge or may not converge at all.

████████████████

In [7]:
# Предсказания на тесте
preds = aml.leader.predict(test_h2o)

# Нам нужен только класс (label), а не вероятности
pred_labels = preds.as_data_frame()['predict']
pred_labels.head()


deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.11/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,predict
0,0
1,0
2,1
3,0
4,0


In [8]:
# Сохраняем предсказания в файл
answers = pd.DataFrame(pred_labels)
answers.columns = ['target']  # Переименовываем колонку, если нужно
answers.to_csv('answers.csv', index=False, header=False)

# Теперь файл готов для отправки!
